<a href="https://colab.research.google.com/github/hoangnguyenbao079/Apple_stocks_predict/blob/main/Apple_Predict.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
import pandas as pd
import datetime as dt
from datetime import date
import matplotlib.pyplot as plt
import yfinance as yf
import numpy as np
import tensorflow as tf
from pyspark.sql import SparkSession
from datetime import date
import time



#Define start day to fetch the dataset from the yahoo finance library

In [10]:
# Bước 1: Tạo SparkSession
spark = SparkSession.builder.appName("StockData").getOrCreate()

# Đặt ngày bắt đầu và ngày kết thúc
START = "2016-01-01"
TODAY = date.today().strftime("%Y-%m-%d")

In [11]:
# Bước 2: Định nghĩa hàm tải dữ liệu từ Yahoo Finance
def load_data(ticker, retries=3, delay=30):
    for attempt in range(retries):
        try:
            data = yf.download(ticker, START, TODAY)
            if data.empty:
                raise ValueError("Empty dataset received.")
            data.reset_index(inplace=True)
            return data
        except Exception as e:
            print(f"Lỗi tải dữ liệu: {e}. Thử lại sau {delay} giây... ({attempt+1}/{retries})")
            time.sleep(delay)
    print("Thất bại sau nhiều lần thử. Vui lòng thử lại sau.")
    return None

In [14]:
# Bước 3: Tải dữ liệu cổ phiếu AAPL
data = load_data('AAPL')
if data is not None:
    # Chuyển dữ liệu từ Pandas DataFrame sang PySpark DataFrame
    df_spark = spark.createDataFrame(data)



[*********************100%***********************]  1 of 1 completed
ERROR:yfinance:
1 Failed download:
ERROR:yfinance:['AAPL']: YFRateLimitError('Too Many Requests. Rate limited. Try after a while.')


Lỗi tải dữ liệu: Empty dataset received.. Thử lại sau 30 giây... (1/3)


[*********************100%***********************]  1 of 1 completed
ERROR:yfinance:
1 Failed download:
ERROR:yfinance:['AAPL']: YFRateLimitError('Too Many Requests. Rate limited. Try after a while.')


Lỗi tải dữ liệu: Empty dataset received.. Thử lại sau 30 giây... (2/3)


[*********************100%***********************]  1 of 1 completed
ERROR:yfinance:
1 Failed download:
ERROR:yfinance:['AAPL']: YFRateLimitError('Too Many Requests. Rate limited. Try after a while.')


Lỗi tải dữ liệu: Empty dataset received.. Thử lại sau 30 giây... (3/3)
Thất bại sau nhiều lần thử. Vui lòng thử lại sau.


In [13]:
# Xoá các cột 'Date' và 'Adj Close'
df_spark = df_spark.drop('Date', 'Adj Close')

# Lấy dữ liệu cột 'Close' về pandas để vẽ
df_plot = df_spark.select("Close").toPandas()

# Vẽ biểu đồ bằng matplotlib
plt.title("Close Price Visualization")
plt.plot(df_plot['Close'])
plt.xlabel("Time (Index)")
plt.ylabel("Close Price")
plt.grid(True)
plt.show()


NameError: name 'df_spark' is not defined